In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
import os
import pandas as pd
from termcolor import colored
import numpy as np
from collections import OrderedDict
from collections import Counter
import matplotlib.pyplot as plt
from natsort import natsorted
import sys
import pickle
import seaborn as sns

# Display pandas df without truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.3g" % x))



## Simple boxplot func

In [3]:
green_diamond = dict(markerfacecolor='g', marker='D')

def simple_boxplot(my_dict, fig_name, yLim, save_flag):
    fig, ax = plt.subplots(figsize=(20, 5), dpi=80)
    ax.boxplot(my_dict.values(), flierprops=green_diamond)
    ax.set_xticklabels(my_dict.keys())
    ax.grid()
    plt.ylim(-1,yLim)
    plt.tight_layout()
    if save_flag:
        plt.savefig(fig_name+'.png')

In [4]:
def set_box_color(bp, edge_color, fill_color):
    plt.setp(bp['boxes'], color=edge_color)
    plt.setp(bp['whiskers'], color=edge_color)
    plt.setp(bp['caps'], color=edge_color)
    plt.setp(bp['medians'], color=edge_color)
    plt.setp(bp['fliers'], color=green_diamond)
#     for flier in bp['fliers']:
#         flier.set(marker='o', color=edge_color, alpha=1)
    
    for patch in bp['boxes']:
        patch.set(facecolor=fill_color)

In [5]:
def boxplot_3dict(dictl, dictc, dictr, plotName):
    ticks = dictr.keys()

    fig, ax = plt.subplots(figsize=(20, 5), dpi=80)

    bpl = ax.boxplot(dictl.values(), patch_artist=True, flierprops=green_diamond, positions=np.array(range(len(dictl.values())))*3.0-0.6, widths=0.4)
    bpr = ax.boxplot(dictr.values(), patch_artist=True, flierprops=green_diamond, positions=np.array(range(len(dictr.values())))*3.0+0.6, widths=0.4)
    bpc = ax.boxplot(dictc.values(), patch_artist=True, flierprops=green_diamond, positions=np.array(range(len(dictc.values())))*3.0, widths=0.4)

    set_box_color(bpl, 'red', 'tan') # colors are from http://colorbrewer2.org/
    set_box_color(bpr, 'blue', 'cyan')
    set_box_color(bpc, 'green', '#BCEE68')

    # draw temporary red and blue lines and use them to create a legend
    plt.plot([], c='red', label='Brownian')
    plt.plot([], c='green', label='Persistent')
    plt.plot([], c='blue', label='Adaptive')
    plt.legend(loc=2)

    # plt.xticks(range(0, 300, 5), ticks)
    # plt.xlim(-3, len(ticks)*3)
    # plt.ylim(0, 80)
    plt.xticks(range(0, len(ticks) * 3, 3), ticks)
    plt.xlim(-2, len(ticks)*3)

    ax.set_xticklabels(dictr.keys())
    ax.set_xlabel("Timeout[s]")
    ax.grid()

#     plt.tight_layout()
    # plt.savefig('boxcompare.png')
    figName = plotName + results_folder.split("/")[-1]+'.png'
    print("Figure name: ", figName)

    plt.savefig(figName)

In [6]:
num_robot = 24
experiment_time = 1800
num_run = 100

In [9]:
results_folder = '/home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/standAlone/results_newWalkParam/brownian/Regions'
# results_folder = '/home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/standAlone/results_newWalkParam/persistent/Regions'
# results_folder = '/home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/standAlone/results_newWalkParam/adaptive/Regions'

# results_folder = '/home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/standAlone/results_newWalkParam/brownian/Mixed'
# results_folder = '/home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/standAlone/results_newWalkParam/persistent/Mixed'
# results_folder = '/home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/standAlone/results_newWalkParam/adaptive/Mixed'

In [10]:
if not os.path.isdir(results_folder):
    print(colored("Error, " + results_folder + " does not exist", 'red'))
else:
    print(colored("OK, " + results_folder + " exists", 'green'))

Error, /home/luigi/Documents/ARGoS/argos_results_scripts/DHTF/standAlone/results_newWalkParam/adaptiveTimeout/Regions/brownian does not exist


In [ ]:
completed_task_label = ["time", "areaID", "creation_time", "completion_time", "type", "kID_on_top", "kIDs"]

## Completed areas per kilobot

In [ ]:
completed_task_dict = {}

timeout = -1
for timeout_folder in natsorted(os.listdir(os.path.join(results_folder))):
    print(colored("Timeout folder:", 'blue'),timeout_folder)
    
    parameters = timeout_folder.split("_")
    
    for param in parameters:
        if param.startswith("timeout"):
#             print(param.split("#")[-1])
            timeout = int(param.split("#")[-1]) * 10
            print("\ttimeout:",timeout)
    
#     if timeout == 20 :
#         break
        
    if timeout == -1 :
        print(colored("\tWARNING: wrong timeout folder", 'green'))
        continue

    completed_task_count = []
    for filename in os.listdir(os.path.join(results_folder, timeout_folder)):
        if filename.endswith("completed_taskLOG.tsv"):
            if not os.path.getsize(os.path.join(results_folder, timeout_folder, filename)) > 0:
                print(colored("\tWARNING, empty file at:" + filename, 'red'))
                continue
#             print('\tfilename: ', filename)
            
            df_completedTask = pd.read_csv(os.path.join(results_folder, timeout_folder, filename), sep="\t", header=None)
            df_completedTask.columns = completed_task_label
            completed_area_kIDs = np.array([]).astype(int)
            for area in df_completedTask["kIDs"].values:
            #     print(np.fromstring(area, dtype=int, sep=','))
                completed_area_kIDs = np.append(completed_area_kIDs, np.fromstring(area, dtype=int, sep=','))
            counter = Counter(completed_area_kIDs)
            
            for i in np.arange(num_robot):
                if i not in counter.keys():
#                     print(i)
                    counter[i] = 0
                    
            completed_task_count += list(counter.values())
#             print(elapsed_timeout_count)
    
    if len(completed_task_count):
        completed_task_dict[str(timeout)] = completed_task_count
    else:
        print(colored("\tWARNING: empty list!!", 'green'))
        completed_task_dict[str(timeout)] = [0] * num_robot * num_run
            


### Brownian

In [ ]:
brownian_dict = completed_task_dict

### Persistent

In [ ]:
persistent_dict = completed_task_dict

### Adaptive

In [ ]:
adaptive_dict = completed_task_dict

In [ ]:
boxplot_3dict(brownian_dict, persistent_dict, adaptive_dict, "completedTaskMean_")

### Persistent

In [ ]:
# saveFigure = False
# figName = "completedTaskMean_persistent_"+results_folder.split("/")[-1]
# print("Figure name: ", figName)
# simple_boxplot(completed_task_dict, figName, 30, saveFigure)

### Brownian

In [ ]:
# saveFigure = False
# figName = "completedTaskMean_brownian_"+results_folder.split("/")[-1]
# print("Figure name: ", figName)
# simple_boxplot(completed_task_dict, figName, 30, saveFigure)

### Adaptive

In [ ]:
# saveFigure = True
# figName = "completedTaskMean_adaptive_"+results_folder.split("/")[-1]
# print("Figure name: ", figName)
# simple_boxplot(completed_task_dict, figName, 30, saveFigure)

## Completed areas in general

In [ ]:
completed_task_dict = {}

timeout = -1
for timeout_folder in natsorted(os.listdir(os.path.join(results_folder))):
    print(colored("Timeout folder:", 'blue'),timeout_folder)
    
    parameters = timeout_folder.split("_")
    
    for param in parameters:
        if param.startswith("timeout"):
#             print(param.split("#")[-1])
            timeout = int(param.split("#")[-1]) * 10
            print("\ttimeout:",timeout)
    
#     if timeout == 20 :
#         break
        
    if timeout == -1 :
        print(colored("\tWARNING: wrong timeout folder", 'green'))
        continue

    completed_task_count = []
    for filename in os.listdir(os.path.join(results_folder, timeout_folder)):
        if filename.endswith("completed_taskLOG.tsv"):
            if not os.path.getsize(os.path.join(results_folder, timeout_folder, filename)) > 0:
                print(colored("\tWARNING, empty file at:" + filename, 'red'))
                completed_task_dict[str(timeout)] = 0
                continue
#             print('\tfilename: ', filename)
            
            df_completedTask = pd.read_csv(os.path.join(results_folder, timeout_folder, filename), sep="\t", header=None)
            completed_task_count += [df_completedTask.shape[0]]
            
    if(len(completed_task_count)!= 100):
        print(colored("\tWARNING, completed_task_count should be 100!", 'red'))
    print(colored("\tlen(completed_task_count)"+str(len(completed_task_count)), 'blue'))
    completed_task_dict[str(timeout)] = completed_task_count
    
            


### Brownian

In [ ]:
brownian_dict = completed_task_dict

### Persistent

In [ ]:
persistent_dict = completed_task_dict

### Adaptvie

In [ ]:
adaptive_dict = completed_task_dict

In [ ]:
boxplot_3dict(brownian_dict, persistent_dict, adaptive_dict, "completedTask_")

### Persistent

In [ ]:
# saveFigure = True
# figureName = "completedTask_persistent_"+results_folder.split("/")[-1]
# print(figureName)
# simple_boxplot(completed_task_dict, figureName, 170, saveFigure)

### Brownian

In [ ]:
# saveFigure = True
# figureName = "completedTask_brownian_"+results_folder.split("/")[-1]
# print(figureName)
# simple_boxplot(completed_task_dict, figureName, 170, saveFigure)

### Adaptive

In [ ]:
# saveFigure = True
# figureName = "completedTask_adaptive_"+results_folder.split("/")[-1]
# print(figureName)
# simple_boxplot(completed_task_dict, figureName, 170, saveFigure)